Here’s a step-by-step guide to set up an EC2 instance using AWS CLI on a Linux machine:

---

### **1. Install and Configure AWS CLI**
1. **Install AWS CLI**:  
   If not already installed, install it:
   ```bash
   sudo apt update && sudo apt install awscli -y   # On Debian/Ubuntu
   ```
   Or for other Linux distributions:
   ```bash
   curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
   unzip awscliv2.zip
   sudo ./aws/install
   ```

2. **Configure AWS CLI**:  
   Set up access keys and default region:
   ```bash
   aws configure
   ```
   Provide:
   - Access Key ID
   - Secret Access Key
   - Default region (e.g., `us-east-1`)
   - Default output format (e.g., `json`)

---

### **2. Create a Key Pair**
A key pair allows secure SSH access to your EC2 instance.
```bash
aws ec2 create-key-pair --key-name MyKeyPair --query 'KeyMaterial' --output text > MyKeyPair.pem
chmod 400 MyKeyPair.pem
```

---

### **3. Create a Security Group**
Define inbound and outbound rules for the instance.
```bash
aws ec2 create-security-group --group-name MySecurityGroup --description "My security group"

# Allow SSH access (port 22) from your IP
aws ec2 authorize-security-group-ingress --group-name MySecurityGroup --protocol tcp --port 22 --cidr YOUR_IP/32

# Allow HTTP traffic (port 80) for web servers
aws ec2 authorize-security-group-ingress --group-name MySecurityGroup --protocol tcp --port 80 --cidr 0.0.0.0/0
```
Replace `YOUR_IP` with your current IP (e.g., `203.0.113.1`).

---

### **4. Launch an EC2 Instance**
Use an AMI (Amazon Machine Image) and instance type:
```bash
aws ec2 run-instances --image-id ami-0abcdef1234567890 --count 1 --instance-type t2.micro \
--key-name MyKeyPair --security-groups MySecurityGroup --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=MyInstance}]'
```

- Replace `ami-0abcdef1234567890` with an AMI ID from your AWS region.
- Instance types like `t2.micro` are part of the free tier (if eligible).

---

### **5. Configure Networking (Optional VPC Setup)**
1. **List Subnets**: Identify a subnet for the instance.
   ```bash
   aws ec2 describe-subnets
   ```
   Use a subnet ID from the output.

2. **Create a Custom VPC (if needed)**:
   ```bash
   aws ec2 create-vpc --cidr-block 10.0.0.0/16
   ```

---

### **6. Verify and Connect**
1. **Get Instance Details**:
   ```bash
   aws ec2 describe-instances --query 'Reservations[*].Instances[*].[InstanceId,State.Name,PublicIpAddress]' --output table
   ```

2. **Connect via SSH**:
   ```bash
   ssh -i MyKeyPair.pem ec2-user@INSTANCE_PUBLIC_IP
   ```

Replace `INSTANCE_PUBLIC_IP` with the public IP address of your instance.

---

### **7. Additional Tips**
- **Stop or Terminate Instances**:
   ```bash
   aws ec2 stop-instances --instance-ids INSTANCE_ID
   aws ec2 terminate-instances --instance-ids INSTANCE_ID
   ```

- **Clean Up**: Ensure to delete unused resources like key pairs, security groups, or VPCs to avoid extra charges.

This setup ensures your EC2 instance has secure access and is configured for basic tasks. Let me know if you need more details on any step!